In [64]:
import pandas as pd
import folium
import seaborn as sns
import json

In [65]:
import matplotlib.pyplot as plt
plt.rc('axes', unicode_minus=False)
plt.rc('font', family='Malgun Gothic')
%matplotlib inline

In [66]:
from IPython.display import set_matplotlib_formats
# 폰트가 선명하게 보이기 위해
set_matplotlib_formats('retina')

In [67]:
df = pd.read_csv("./상가(상권)정보_의료기관_201909.csv", low_memory=False)

In [68]:
df_nowon = df[df["시군구명"] == "노원구"].copy()
df_medical = df_nowon[df_nowon["상권업종중분류명"] == "병원"]

In [83]:
# 동별 지도화
dongDf = pd.read_csv('./old_people.csv', index_col='동')  
dongDf.columns = dongDf.columns.map(str); dongDf
dongDevided = json.load(open( './map.json', encoding = 'utf-8'))
df_park = pd.read_csv("./park.csv")

In [84]:
p_loc = df_park[["위도", "경도"]].values[:len(df_park)].tolist()
p_loc

[[37.68068531121212, 127.05751881235064],
 [37.65091867047456, 127.06467364118492],
 [37.64939717395541, 127.05751013933734],
 [37.64787845700477, 127.06342891234952],
 [37.64407536744455, 127.05852676817248],
 [37.66919897583556, 127.05053601050264],
 [37.665960685841, 127.05431231235016],
 [37.66703458252295, 127.06101076817338],
 [37.661333627148345, 127.0641175393377],
 [37.64913614305768, 127.07097516817268],
 [37.647363178608686, 127.07538951234947],
 [37.65399080229555, 127.060775699713],
 [37.64001988698929, 127.06580731234924],
 [37.64060771117623, 127.06704395283104],
 [37.64873000673272, 127.0663137951605],
 [37.65818607846887, 127.0760103663254],
 [37.64096041485212, 127.0768529393371],
 [37.638847306241, 127.07402628166618],
 [37.644978550651714, 127.07432533933718],
 [37.62532544927705, 127.05739629885473],
 [37.639185840973376, 127.04678624840184],
 [37.6314542000589, 127.04885410127326],
 [37.62511087686544, 127.06928371755384],
 [37.620879727066544, 127.08641011234867]

In [85]:
dongMap = folium.Map(location=[df_medical["위도"].mean(), df_medical["경도"].mean()], zoom_start = 12.5, 
                   tiles = 'openstreetmap'); dongMap
fmap = folium.Choropleth(geo_data = dongDevided,
                         data = dongDf,
                         columns = ['행정구역', '명'],
                         key_on = 'feature.properties.adm_nm', fill_color='YlOrRd',
                         fill_opacity = 0.7, line_opacity = 0.3,
                         legend_name = '노인 인구수',
                         highlight=True
                         
                        ).add_to(dongMap)

folium.LayerControl().add_to(dongMap)

# m = folium.Map(location=[df_medical["위도"].mean(), df_medical["경도"].mean()], zoom_start = 12.5)
# 병원 
loc = df_medical[["위도", "경도"]].values[:len(df_medical)].tolist()
for i in range(len(df_medical)):
    folium.Circle(location=loc[i], radius=5, color="blue", fill = True).add_to(dongMap)

p_loc = df_park[["위도", "경도"]].values[:len(df_park)].tolist()

for i in range(len(df_park)):
    folium.Marker(location = p_loc[i], tooltip = df_park['공원명'][i],icon = folium.Icon(color = 'blue', icon = 'info-sign')).add_to(dongMap)

fmap.geojson.zoom_on_click = False
fmap.geojson.add_child(
     folium.features.GeoJsonTooltip(['adm_nm'],labels = False)
)
dongMap.save('./dongMap.html'); dongMap